# CIS 194: Homework 5

On day one of your new job as a software engineer, you’ve been
asked to program the brains of the company’s new blockbuster product: a calculator. But this isn’t just any calculator! Extensive focus
group analysis has revealed that what people really want out of their
calculator is something that can add and multiply integers. Anything
more just clutters the interface.

Your boss has already started by modeling the domain with the
following data type of arithmetic expressions:

```haskell
data ExprT = Lit Integer
           | Add ExprT ExprT
           | Mul ExprT ExprT
  deriving (Show, Eq)
```

This type is capable of representing expressions involving integer
constants, addition, and multiplication. For example, the expression
(2 + 3) × 4 would be represented by the value

```haskell
Mul (Add (Lit 2) (Lit 3)) (Lit 4).
```

Your boss has already provided the definition of ExprT in ExprT.hs,
so as usual you just need to add import ExprT to the top of your file.
However, this is where your boss got stuck.

## Exercise 1

Write Version 1 of the calculator: an evaluator for ExprT, with the
signature

```haskell
eval :: ExprT -> Integer
```

For example:

```haskell
eval (Mul (Add (Lit 2) (Lit 3)) (Lit 4)) == 20.
```

In [1]:
:l ExprT
:l Parser

In [2]:
eval :: ExprT -> Integer
eval (Add m n) = (+) (eval m) (eval n)
eval (Mul m n) = (*) (eval m) (eval n)
eval (Lit x) = x

In [3]:
eval (Mul (Add (Lit 2) (Lit 3)) (Lit 4)) == 20

True

## Exercise 2

The UI department has internalized the focus group data and is
ready to synergize with you. They have developed the front-facing
user-interface: a parser that handles the textual representation of the
selected language. They have sent you the module `Parser.hs`, which
exports `parseExp`, a parser for arithmetic expressions. If you pass
the constructors of `ExprT` to it as arguments, it will convert Strings
representing arithmetic expressions into values of type `ExprT`

For example:

```
*Calc> parseExp Lit Add Mul "(2+3)*4"
Just (Mul (Add (Lit 2) (Lit 3)) (Lit 4))
*Calc> parseExp Lit Add Mul "2+3*4"
Just (Add (Lit 2) (Mul (Lit 3) (Lit 4)))
*Calc> parseExp Lit Add Mul "2+3*"
Nothing
```

Leverage the assets of the UI team to implement the value-added
function

```haskell
evalStr :: String -> Maybe Integer
```

which evaluates arithmetic expressions given as a `String`, producing `Nothing` for inputs which are not well-formed expressions, and
`Just n` for well-formed inputs that evaluate to *n*.

In [4]:
evalStr :: String -> Maybe Integer
evalStr = fmap eval . parseExp Lit Add Mul

In [5]:
evalStr "(2+3)*4"

Just 20

In [6]:
evalStr "2+3*4"

Just 14

In [7]:
evalStr "2+3*"

Nothing

## Exercise 3

Good news! Early customer feedback indicates that people really
do love the interface! Unfortunately, there seems to be some disagreement over exactly how the calculator should go about its calculating
business. The problem the software department (*i.e.* you) has is that
while `ExprT` is nice, it is also rather inflexible, which makes catering
to diverse demographics a bit clumsy. You decide to abstract away
the properties of `ExprT` with a type class.

Create a type class called `Expr` with three methods called lit, add,
and mul which parallel the constructors of `ExprT`. Make an instance of
Expr for the `ExprT` type, in such a way that

```haskell
mul (add (lit 2) (lit 3)) (lit 4) :: ExprT
 ==  Mul (Add (Lit 2) (Lit 3)) (Lit 4)
```

Think carefully about what types lit, add, and mul should have. It
may be helpful to consider the types of the `ExprT` constructors, which
you can find out by typing 

In [8]:
:t Lit

Lit :: Integer -> ExprT

at the ghci prompt.

In [9]:
class Expr a where
  lit :: Integer -> a
  add :: a -> a -> a
  mul :: a -> a -> a

instance Expr ExprT where
  lit = Lit
  add = Add
  mul = Mul

In [10]:
(mul (add (lit 2) (lit 3)) (lit 4) :: ExprT) == Mul (Add (Lit 2) (Lit 3)) (Lit 4)

True

**Remark.** Take a look at the type of the foregoing example expression:

In [11]:
:t  mul (add (lit 2) (lit 3)) (lit 4)

mul (add (lit 2) (lit 3)) (lit 4) :: forall {a}. Expr a => a

What does this mean? The expression `mul (add (lit 2) (lit 3)) (lit 4)` has *any type* which is an instance of the `Expr` type class. So writing it
by itself is ambiguous: GHC doesn’t know what concrete type you
want to use, so it doesn’t know which implementations of mul, add,
and lit to pick.

One way to resolve the ambiguity is by giving an explicit type
signature, as in the above example. Another way is by using such an
expression as part of some larger expression so that the context in
which it is used determines the type. For example, we may write a
function reify as follows:

In [12]:
reify :: ExprT -> ExprT
reify = id

To the untrained eye it may look like reify does no actual work!
But its real purpose is to constrain the type of its argument to ExprT.
Now we can write things like

In [13]:
reify $ mul (add (lit 2) (lit 3)) (lit 4)

Mul (Add (Lit 2) (Lit 3)) (Lit 4)

at the ghci prompt.

## Exercise 4

The marketing department has gotten wind of just how flexible
the calculator project is and has promised custom calculators to some
big clients. As you noticed after the initial roll-out, everyone loves the
interface, but everyone seems to have their own opinion on what the
**semantics** should be. Remember when we wrote `ExprT` and thought
that addition and multiplication of integers was pretty cut and dried?
Well, it turns out that some big clients want customized calculators
with behaviors that they have decided are right for them.

The point of our `Expr` type class is that we can now write down
arithmetic expressions **once** and have them interpreted in various
ways just by using them at various types.

Make instances of `Expr` for each of the following types:

- Integer — works like the original calculator
- Bool — every literal value less than or equal to 0 is interpreted as **False**, and all positive **Integers**
are interpreted as **True**; “addition” is logical or,
“multiplication” is logical and
- MinMax — “addition” is taken to be the max function, while
“multiplication” is the min function
- Mod7 — all values should be in the range `0 . . . 6`, and
all arithmetic is done modulo *7*; for example,
`5 + 3 = 1`.

The last two variants work with Integers internally, but in order
to provide different instances, we wrap those Integers in newtype
wrappers. These are used just like the data constructors we’ve seen
before.

In [14]:
newtype MinMax = MinMax Integer deriving (Eq, Show)
newtype Mod7 = Mod7 Integer deriving (Eq, Show)

In [15]:
instance Expr Integer where
  lit = id
  add = (+)
  mul = (*)

instance Expr Bool where
  lit = (> 0)
  add = (||)
  mul = (&&)

instance Expr MinMax where
  lit = MinMax
  add (MinMax a) (MinMax b) = MinMax $ max a b
  mul (MinMax a) (MinMax b) = MinMax $ min a b

instance Expr Mod7 where
  lit = Mod7 . (`mod` 7)
  add (Mod7 a) (Mod7 b) = Mod7 $ (+) a b `mod` 7
  mul (Mod7 a) (Mod7 b) = Mod7 $ (*) a b `mod` 7

Once done, the following code should demonstrate our family of
calculators:

In [16]:
testExp :: Expr a => Maybe a
testExp = parseExp lit add mul "(3 * -4) + 5"

In [17]:
testInteger = testExp :: Maybe Integer
testBool = testExp :: Maybe Bool
testMM = testExp :: Maybe MinMax
testSat = testExp :: Maybe Mod7

Try printing out each of those tests in ghci to see if things are
working. It’s great how easy it is for us to swap in new semantics for
the same syntactic expression!

In [18]:
testInteger

Just (-7)

In [19]:
testBool

Just True

In [20]:
testMM

Just (MinMax 5)

In [21]:
testSat

Just (Mod7 0)